# Import Modules


In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
import datetime

In [ ]:
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git #run this one
#!pip install stable-baselines3[extra]

In [5]:
from gym.utils import seeding
import gym
from gym import spaces
from stable_baselines3.common.vec_env import DummyVecEnv

In [6]:
from finrl import config
from finrl.finrl_meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.finrl_meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.finrl_meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.drl_agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline

from pprint import pprint

/usr/local/lib/python3.7/dist-packages/pyfolio/pos.py:27: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  'Module "zipline.assets" not found; multipliers will not be applied'


In [7]:
import itertools
import sys

Stable-baseline imports

In [8]:
import stable_baselines3
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env

In [55]:
import torch
%matplotlib inline
!pip install pyyaml==5.4.1
import plotly.express as px

     |████████████████████████████████| 636 kB 3.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


TypeError: ignored

In [ ]:
#!pip install PyPortfolioOpt

In [57]:
from pypfopt import plotting
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
import pandas as pd
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt import objective_functions
unique_tic = trade.tic.unique()
unique_trade_date = trade.date.unique()

# Download Stock Data from Yahoo Finance

In [9]:
config.START_DATE

'2009-01-01'

In [10]:
config.END_DATE

'2021-10-31'

In [11]:
tickers = ['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 
          'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW'] #ticker types defined

In [12]:
df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-10-31',
                     ticker_list = tickers).fetch_data() #get data from Yahoo Downloader

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [13]:
df.shape #shape of data frame 

(94331, 8)

In [14]:
df.head() #head of data frame

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.775245,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,45.228069,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.579447,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,32.318985,7117200,CAT,4


In [15]:
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d') #convert date to datetime object

In [16]:
df.sort_values(['date','tic'],ignore_index=True).head() #sort according to date ticker combination

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.775245,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,45.228069,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.579447,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,32.318985,7117200,CAT,4


# Preprocess Data

In [17]:
'''
TECHNICAL_INDICATORS_LIST = [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
]
'''
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                    use_vix=False,
                    use_turbulence=True,
                    user_defined_feature = False) #inclided techincak indicators as features


In [18]:
df_processed = fe.preprocess_data(df)

Successfully added technical indicators
Successfully added turbulence index


In [19]:
df_processed #preprocessed data frame

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,3.067143,3.251429,3.041429,2.775245,746015200,AAPL,4,0.000000,2.999449,2.668167,100.000000,66.666667,100.000000,2.775245,2.775245,0.000000
1,2009-01-02,58.590000,59.080002,57.750000,45.228069,6547900,AMGN,4,0.000000,2.999449,2.668167,100.000000,66.666667,100.000000,45.228069,45.228069,0.000000
2,2009-01-02,18.570000,19.520000,18.400000,15.579447,10955700,AXP,4,0.000000,2.999449,2.668167,100.000000,66.666667,100.000000,15.579447,15.579447,0.000000
3,2009-01-02,42.799999,45.560001,42.779999,33.941101,7010200,BA,4,0.000000,2.999449,2.668167,100.000000,66.666667,100.000000,33.941101,33.941101,0.000000
4,2009-01-02,44.910000,46.980000,44.709999,32.318985,7117200,CAT,4,0.000000,2.999449,2.668167,100.000000,66.666667,100.000000,32.318985,32.318985,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93665,2021-10-29,454.410004,461.390015,453.059998,457.618835,2496700,UNH,4,13.088377,469.819477,374.630236,67.009701,145.383219,52.064871,415.194161,414.296775,21.954439
93666,2021-10-29,209.210007,213.669998,208.539993,211.054474,14329800,V,4,-2.396987,239.187034,211.712324,40.256694,-230.770174,39.627801,225.309486,227.454724,21.954439
93667,2021-10-29,52.500000,53.049999,52.410000,52.359169,17758800,VZ,4,-0.190820,53.795641,50.555179,46.685018,-39.092476,12.569826,52.440649,53.111298,21.954439
93668,2021-10-29,46.860001,47.279999,46.770000,46.105030,4998000,WBA,4,-0.188551,49.079663,44.995378,45.913458,-97.149514,0.268701,47.068244,47.621484,21.954439


In [20]:
df_processed['date'] = df_processed['date'].astype(str) #convert to string temporarily for concatination purposes

In [21]:
ticker_list = df_processed["tic"].unique().tolist() #get ticker types

In [22]:
date_list = list(pd.date_range(df_processed['date'].min(),df_processed['date'].max()).astype(str)) #get dates

In [23]:
date_ticker_list = list(itertools.product(date_list, ticker_list)) #combine them 

In [24]:
df_processed_full = pd.DataFrame(date_ticker_list,columns=["date","tic"]).merge(df_processed,on=["date","tic"],how="left") #apply left join with that combination 

In [25]:
df_processed_full['date'] = pd.to_datetime(df_processed_full['date'],format='%Y-%m-%d') #back to datetime format

In [26]:
df_processed_full = df_processed_full[df_processed_full['date'].isin(df_processed['date'])] #keep only actual data by matching the dates

In [27]:
df_processed_full = df_processed_full.sort_values(['date','tic']) #sort by date-ticker combination

In [28]:
df_processed_full = df_processed_full.fillna(0) #fill the missing data with 0

In [29]:
df_processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.775245,746015200.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,2.775245,2.775245,0.0
1,2009-01-02,AMGN,58.590000,59.080002,57.750000,45.228069,6547900.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,45.228069,45.228069,0.0
2,2009-01-02,AXP,18.570000,19.520000,18.400000,15.579447,10955700.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,15.579447,15.579447,0.0
3,2009-01-02,BA,42.799999,45.560001,42.779999,33.941101,7010200.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,33.941101,33.941101,0.0
4,2009-01-02,CAT,44.910000,46.980000,44.709999,32.318985,7117200.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,32.318985,32.318985,0.0
5,2009-01-02,CRM,8.025000,8.550000,7.912500,8.505000,4069200.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,8.505000,8.505000,0.0
6,2009-01-02,CSCO,16.410000,17.000000,16.250000,12.349072,40980600.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,12.349072,12.349072,0.0
7,2009-01-02,CVX,74.230003,77.300003,73.580002,45.176243,13695900.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,45.176243,45.176243,0.0
8,2009-01-02,DIS,22.760000,24.030001,22.500000,20.597494,9796600.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,20.597494,20.597494,0.0
9,2009-01-02,GS,84.019997,87.620003,82.190002,71.168251,14088500.0,4.0,0.0,2.999449,2.668167,100.0,66.666667,100.0,71.168251,71.168251,0.0


In [30]:
df_processed_full=df_processed_full.sort_values(['date','tic'],ignore_index=True)
df_processed_full.index = df_processed_full.date.factorize()[0]

In [31]:
#include covariance of stocks as feature depending on 1 year data 
cov_list = []
return_list = []

# look back is one year
lookback=252
for i in range(lookback,len(df_processed_full.index.unique())):
  data_lookback = df_processed_full.loc[i-lookback:i,:]
  price_lookback=data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
  return_lookback = price_lookback.pct_change().dropna()
  return_list.append(return_lookback)

  covs = return_lookback.cov().values 
  cov_list.append(covs)

In [32]:
df_cov = pd.DataFrame({'date':df_processed_full.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
df_processed_full = df_processed_full.merge(df_cov, on='date')
df_processed_full = df_processed_full.sort_values(['date','tic']).reset_index(drop=True)

In [33]:
df_processed_full

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,cov_list,return_list
0,2010-01-04,AAPL,7.622500,7.660714,7.585000,6.544687,493729600.0,0.0,0.120073,6.609204,5.627351,62.133174,168.777455,33.760635,6.122819,6.069180,0.000000,"[[0.00044306061076910435, 0.000136980052220275...",tic AAPL AMGN AXP ...
1,2010-01-04,AMGN,56.630001,57.869999,56.560001,44.254356,5277400.0,0.0,0.247149,44.727425,42.036266,52.849992,86.594483,6.350919,43.374945,43.293035,0.000000,"[[0.00044306061076910435, 0.000136980052220275...",tic AAPL AMGN AXP ...
2,2010-01-04,AXP,40.810001,41.099998,40.389999,34.145287,6894300.0,0.0,0.300254,35.104355,32.706430,56.779357,1.197230,11.537387,34.028193,32.383233,0.000000,"[[0.00044306061076910435, 0.000136980052220275...",tic AAPL AMGN AXP ...
3,2010-01-04,BA,55.720001,56.389999,54.799999,43.777550,6186700.0,0.0,0.501040,44.009588,41.898202,58.804998,81.205407,10.840906,42.319076,40.752999,0.000000,"[[0.00044306061076910435, 0.000136980052220275...",tic AAPL AMGN AXP ...
4,2010-01-04,CAT,57.650002,59.189999,57.509998,42.120888,7325600.0,0.0,0.079973,42.540558,40.531464,55.292750,49.634391,8.534279,41.704589,41.306262,0.000000,"[[0.00044306061076910435, 0.000136980052220275...",tic AAPL AMGN AXP ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86357,2021-10-29,UNH,454.410004,461.390015,453.059998,457.618835,2496700.0,4.0,13.088377,469.819477,374.630236,67.009701,145.383219,52.064871,415.194161,414.296775,21.954439,"[[0.0002692158476761782, 5.192277497278833e-05...",tic AAPL AMGN AXP ...
86358,2021-10-29,V,209.210007,213.669998,208.539993,211.054474,14329800.0,4.0,-2.396987,239.187034,211.712324,40.256694,-230.770174,39.627801,225.309486,227.454724,21.954439,"[[0.0002692158476761782, 5.192277497278833e-05...",tic AAPL AMGN AXP ...
86359,2021-10-29,VZ,52.500000,53.049999,52.410000,52.359169,17758800.0,4.0,-0.190820,53.795641,50.555179,46.685018,-39.092476,12.569826,52.440649,53.111298,21.954439,"[[0.0002692158476761782, 5.192277497278833e-05...",tic AAPL AMGN AXP ...
86360,2021-10-29,WBA,46.860001,47.279999,46.770000,46.105030,4998000.0,4.0,-0.188551,49.079663,44.995378,45.913458,-97.149514,0.268701,47.068244,47.621484,21.954439,"[[0.0002692158476761782, 5.192277497278833e-05...",tic AAPL AMGN AXP ...


In [ ]:
#df_processed_full.to_csv('30_dow_data.csv')

In [ ]:
#from google.colab import files
#df_processed_full = files.upload()

# Environment Design

The action space describes the allowed portfolio weights that the agent interacts with the environment. Each element in the portfolio weights vector is non-negative and no more than 100%. Also, the sum of elements in each portfolio weight should equal to 100%.

In [34]:
train = data_split(df_processed_full, '2009-01-01','2020-07-01') #train
trade = data_split(df_processed_full, '2020-07-01','2021-10-31') #trade
print(len(train))
print(len(trade))

76589
9773


In [35]:
train.tail(3)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,cov_list,return_list
2640,2020-06-30,VZ,54.919998,55.290001,54.360001,50.990154,17414800.0,1.0,-0.442434,54.574966,49.322676,48.097037,-50.972374,8.508886,51.633274,52.091341,12.918707,"[[0.0006523840352890155, 0.0004061941184846327...",tic AAPL AMGN AXP ...
2640,2020-06-30,WBA,42.119999,42.580002,41.759998,39.494282,4782100.0,1.0,-0.084972,43.109827,36.915701,48.830192,-14.474246,1.500723,39.594902,39.392491,12.918707,"[[0.0006523840352890155, 0.0004061941184846327...",tic AAPL AMGN AXP ...
2640,2020-06-30,WMT,119.220001,120.129997,118.540001,116.994621,6836400.0,1.0,-0.893233,120.371805,114.363676,48.159675,-69.914655,3.847271,118.672997,120.623193,12.918707,"[[0.0006523840352890155, 0.0004061941184846327...",tic AAPL AMGN AXP ...


In [36]:
trade.head(3)

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence,cov_list,return_list
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,90.036705,110737200.0,2.0,3.019033,92.835519,80.297755,62.807125,107.489613,29.730532,84.057088,77.831730,53.068175,"[[0.0006524293048639158, 0.0004047210470333797...",tic AAPL AMGN AXP ...
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,242.083481,6575800.0,2.0,3.665610,234.264610,201.821542,61.279650,271.571322,46.806139,216.584932,217.666554,53.068175,"[[0.0006524293048639158, 0.0004047210470333797...",tic AAPL AMGN AXP ...
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.347778,3301000.0,2.0,-0.390268,110.737395,88.008445,48.504818,-66.334515,3.142448,97.520676,90.952974,53.068175,"[[0.0006524293048639158, 0.0004047210470333797...",tic AAPL AMGN AXP ...


In [37]:
class StockPortfolioEnv(gym.Env):
    """A portfolio allocation environment for OpenAI gym

    Attributes
    ----------
        df: DataFrame
            input data
        stock_dim : int
            number of unique stocks
        hmax : int
            maximum number of shares to trade
        initial_amount : int
            start money
        transaction_cost_pct: float
            transaction cost percentage per trade
        reward_scaling: float
            scaling factor for reward, good for training
        state_space: int
            the dimension of input features
        action_space: int
            equals stock dimension
        tech_indicator_list: list
            a list of technical indicator names
        turbulence_threshold: int
            a threshold to control risk aversion
        day: int
            an increment number to control date

    Methods
    -------
    _sell_stock()
        perform sell action based on the sign of the action
    _buy_stock()
        perform buy action based on the sign of the action
    step()
        at each step the agent will return actions, then 
        we will calculate the reward, and return the next observation.
    reset()
        reset the environment
    render()
        use render to return other functions
    save_asset_memory()
        return account value at each time step
    save_action_memory()
        return actions/positions at each time step
        

    """
    metadata = {'render.modes': ['human']}

    def __init__(self, 
                df,
                stock_dim,
                hmax,
                initial_amount,
                transaction_cost_pct,
                reward_scaling,
                state_space,
                action_space,
                tech_indicator_list,
                turbulence_threshold=None,
                lookback=252,
                day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.lookback=lookback
        self.df = df
        self.stock_dim = stock_dim
        self.hmax = hmax
        self.initial_amount = initial_amount
        self.transaction_cost_pct =transaction_cost_pct
        self.reward_scaling = reward_scaling
        self.state_space = state_space
        self.action_space = action_space
        self.tech_indicator_list = tech_indicator_list

        # action_space normalization and shape is self.stock_dim
        self.action_space = spaces.Box(low = 0, high = 1,shape = (self.action_space,)) 
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape = (self.state_space+len(self.tech_indicator_list),self.state_space))

        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold        
        # initalize state: inital portfolio return + individual stock return + individual weights
        self.portfolio_value = self.initial_amount

        # memorize portfolio value each step
        self.asset_memory = [self.initial_amount]
        # memorize portfolio return each step
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]]

        
    def step(self, actions):
        self.terminal = self.day >= len(self.df.index.unique())-1

        if self.terminal:
            df = pd.DataFrame(self.portfolio_return_memory)
            df.columns = ['daily_return']
            plt.plot(df.daily_return.cumsum(),'r')
            plt.savefig('results/cumulative_reward.png')
            plt.close()
            
            plt.plot(self.portfolio_return_memory,'r')
            plt.savefig('results/rewards.png')
            plt.close()

            print("=================================")
            print("begin_total_asset:{}".format(self.asset_memory[0]))           
            print("end_total_asset:{}".format(self.portfolio_value))

            df_daily_return = pd.DataFrame(self.portfolio_return_memory)
            df_daily_return.columns = ['daily_return']
            if df_daily_return['daily_return'].std() !=0:
              sharpe = (252**0.5)*df_daily_return['daily_return'].mean()/ \
                       df_daily_return['daily_return'].std()
              print("Sharpe: ",sharpe)
            print("=================================")
            
            return self.state, self.reward, self.terminal,{}

        else:
            weights = self.softmax_normalization(actions) 
            self.actions_memory.append(weights)
            last_day_memory = self.data

            #load next state
            self.day += 1
            self.data = self.df.loc[self.day,:]
            self.covs = self.data['cov_list'].values[0]
            self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
            portfolio_return = sum(((self.data.close.values / last_day_memory.close.values)-1)*weights)
            log_portfolio_return = np.log(sum((self.data.close.values / last_day_memory.close.values)*weights))
            # update portfolio value
            new_portfolio_value = self.portfolio_value*(1+portfolio_return)
            self.portfolio_value = new_portfolio_value

            # save into memory
            self.portfolio_return_memory.append(portfolio_return)
            self.date_memory.append(self.data.date.unique()[0])            
            self.asset_memory.append(new_portfolio_value)

            # the reward is the new portfolio value or end portfolo value
            self.reward = new_portfolio_value
            

        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [self.initial_amount]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        # load states
        self.covs = self.data['cov_list'].values[0]
        self.state =  np.append(np.array(self.covs), [self.data[tech].values.tolist() for tech in self.tech_indicator_list ], axis=0)
        self.portfolio_value = self.initial_amount
        #self.cost = 0
        #self.trades = 0
        self.terminal = False 
        self.portfolio_return_memory = [0]
        self.actions_memory=[[1/self.stock_dim]*self.stock_dim]
        self.date_memory=[self.data.date.unique()[0]] 
        return self.state
    
    def render(self, mode='human'):
        return self.state
        
    def softmax_normalization(self, actions):
        numerator = np.exp(actions)
        denominator = np.sum(np.exp(actions))
        softmax_output = numerator/denominator
        return softmax_output

    
    def save_asset_memory(self):
        date_list = self.date_memory
        portfolio_return = self.portfolio_return_memory
        #print(len(date_list))
        #print(len(asset_list))
        df_account_value = pd.DataFrame({'date':date_list,'daily_return':portfolio_return})
        return df_account_value

    def save_action_memory(self):
        # date and close price length must match actions length
        date_list = self.date_memory
        df_date = pd.DataFrame(date_list)
        df_date.columns = ['date']
        
        action_list = self.actions_memory
        df_actions = pd.DataFrame(action_list)
        df_actions.columns = self.data.tic.values
        df_actions.index = df_date.date
        #df_actions = pd.DataFrame({'date':date_list,'actions':action_list})
        return df_actions

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def get_sb_env(self):
        e = DummyVecEnv([lambda: self])
        obs = e.reset()
        return e, obs

In [38]:
'''
TECHNICAL_INDICATORS_LIST = [
    "macd",
    "boll_ub",
    "boll_lb",
    "rsi_30",
    "cci_30",
    "dx_30",
    "close_30_sma",
    "close_60_sma",
]
'''

'\nTECHNICAL_INDICATORS_LIST = [\n    "macd",\n    "boll_ub",\n    "boll_lb",\n    "rsi_30",\n    "cci_30",\n    "dx_30",\n    "close_30_sma",\n    "close_60_sma",\n]\n'

In [39]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension 

In [40]:
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 29


In [41]:
feature_dimension = len(config.TECHNICAL_INDICATORS_LIST)

In [42]:
print(f"Feature Dimension: {feature_dimension}")

Feature Dimension: 8


In [37]:
#initial_list = [1000000] + ([0] * 29)# the initial total asset is calculated by cash + sum (num_share_stock_i * price_stock_i)

In [43]:
env_kwargs = {
    "hmax": 100, #maximum number of shares to trade
    "initial_amount": 1000000, # initial cash
    "transaction_cost_pct": 0.001, # transaction cost percentage
    "state_space": state_space, # number of unique stocks 
    "stock_dim": stock_dimension, # number of unique stocks
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, # technical indicators
    "action_space": stock_dimension, # number of stocks in training data
    "reward_scaling": 1e-1  #hyperparameter
}

In [44]:
e_train_gym = StockPortfolioEnv(df = train, **env_kwargs)

In [45]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


# Model Layer

Model 1)  A2C

In [46]:
agent_a2c = DRLAgent(env = env_train)

A2C_PARAMS = {"n_steps": 10, "ent_coef": 0.005, "learning_rate": 0.0004}
model_a2c = agent_a2c.get_model(model_name="a2c",model_kwargs = A2C_PARAMS)

{'n_steps': 10, 'ent_coef': 0.005, 'learning_rate': 0.0004}
Using cpu device


In [49]:
trained_a2c = agent_a2c.train_model(model=model_a2c, tb_log_name='a2c',
                                total_timesteps=40000)

-------------------------------------
| time/                 |           |
|    fps                | 392       |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 1000      |
| train/                |           |
|    entropy_loss       | -41       |
|    explained_variance | 1.19e-07  |
|    learning_rate      | 0.0004    |
|    n_updates          | 627       |
|    policy_loss        | 4.02e+08  |
|    reward             | 2003307.6 |
|    std                | 0.994     |
|    value_loss         | 1.4e+14   |
-------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 393       |
|    iterations         | 200       |
|    time_elapsed       | 5         |
|    total_timesteps    | 2000      |
| train/                |           |
|    entropy_loss       | -41       |
|    explained_variance | 0         |
|    learning_rate      | 0.0004    |
|    n_updat

Model 2) PPO

In [50]:
agent_ppo = DRLAgent(env = env_train)

PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.005,
    "learning_rate": 0.001,
    "batch_size": 128,
}
model_ppo = agent_ppo.get_model("ppo",model_kwargs = PPO_PARAMS)

{'n_steps': 2048, 'ent_coef': 0.005, 'learning_rate': 0.001, 'batch_size': 128}
Using cpu device


In [51]:
trained_ppo = agent_ppo.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=40000)

----------------------------------
| time/              |           |
|    fps             | 443       |
|    iterations      | 1         |
|    time_elapsed    | 4         |
|    total_timesteps | 2048      |
| train/             |           |
|    reward          | 3540627.8 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4520283.504409047
Sharpe:  0.9158629969516009
---------------------------------------
| time/                   |           |
|    fps                  | 402       |
|    iterations           | 2         |
|    time_elapsed         | 10        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -41.1     |
|    explained_variance   | 0         |
|    learning_rate        | 0.001     |
|    loss                 | 6.05e+14  |
|    n_updates            | 10       

Model 3) DDPG

In [89]:
agent_ddpg = DRLAgent(env = env_train)
DDPG_PARAMS = {"batch_size": 128, "buffer_size": 50000, "learning_rate": 0.001}


model_ddpg = agent_ddpg.get_model("ddpg",model_kwargs = DDPG_PARAMS)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [90]:
trained_ddpg = agent_ddpg.train_model(model=model_ddpg, 
                             tb_log_name='ddpg',
                             total_timesteps=50000)

begin_total_asset:1000000
end_total_asset:4308359.561581904
Sharpe:  0.8884931783490068
begin_total_asset:1000000
end_total_asset:4237070.286380423
Sharpe:  0.8831671010448424
begin_total_asset:1000000
end_total_asset:4237070.286380423
Sharpe:  0.8831671010448424
begin_total_asset:1000000
end_total_asset:4237070.286380423
Sharpe:  0.8831671010448424
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 21        |
|    time_elapsed    | 487       |
|    total_timesteps | 10564     |
| train/             |           |
|    actor_loss      | -6.5e+07  |
|    critic_loss     | 3.28e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 7923      |
|    reward          | 4237070.5 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4237070.286380423
Sharpe:  0.8831671010448424
begin_total_asset:1000000
end_total_asset:4237070.286380423
Sharpe:  0.8831671010448424
begin_total_asset

KeyboardInterrupt: ignored

Model 4) TD3

In [92]:
agent_td3 = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100, 
              "buffer_size": 1000000, 
              "learning_rate": 0.001}

model_td3 = agent_td3.get_model("td3",model_kwargs = TD3_PARAMS)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


In [93]:
trained_td3 = agent_td3.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=30000)

begin_total_asset:1000000
end_total_asset:4653167.605480633
Sharpe:  0.9378037898968495
begin_total_asset:1000000
end_total_asset:4472017.868082224
Sharpe:  0.900196020127529
begin_total_asset:1000000
end_total_asset:4472017.868082224
Sharpe:  0.900196020127529
begin_total_asset:1000000
end_total_asset:4472017.868082224
Sharpe:  0.900196020127529
----------------------------------
| time/              |           |
|    episodes        | 4         |
|    fps             | 23        |
|    time_elapsed    | 443       |
|    total_timesteps | 10564     |
| train/             |           |
|    actor_loss      | -3.58e+07 |
|    critic_loss     | 2.31e+12  |
|    learning_rate   | 0.001     |
|    n_updates       | 7923      |
|    reward          | 4472018.0 |
----------------------------------
begin_total_asset:1000000
end_total_asset:4472017.868082224
Sharpe:  0.900196020127529


KeyboardInterrupt: ignored

Model 5) SAC

In [ ]:
agent_sac = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0003,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent_sac.get_model("sac",model_kwargs = SAC_PARAMS)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0003, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [ ]:
trained_sac = agent_sac.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=50000)

Model 6) Conventional Models

# Evaluation Layer (Back-Testing)

Reinforcement learning models test results

In [52]:
#trade = data_split(df_processed_full, '2020-07-01','2021-10-31') #trade
e_trade_gym = StockPortfolioEnv(df = trade, **env_kwargs)

In [59]:
df_daily_return_a2c, df_actions_a2c = DRLAgent.DRL_prediction(model=trained_a2c,
                        environment = e_trade_gym)

begin_total_asset:1000000
end_total_asset:1390529.8622496426
Sharpe:  1.8444092568541044
hit end!


In [60]:
df_daily_return_ppo, df_actions_ppo = DRLAgent.DRL_prediction(model=trained_ppo,
                        environment = e_trade_gym)

begin_total_asset:1000000
end_total_asset:1412097.0670661693
Sharpe:  1.9503398482773522
hit end!


In [ ]:
df_daily_return_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(model=trained_ddpg,
                        environment = e_trade_gym) #not trained yet

In [ ]:
df_daily_return_td3, df_actions_td3 = DRLAgent.DRL_prediction(model=trained_td3,
                        environment = e_trade_gym) #not trained yet

In [ ]:
df_daily_return_sac, df_actions_sac = DRLAgent.DRL_prediction(model=trained_sac,
                        environment = e_trade_gym) #not trained yet

Conventional models test results

In [ ]:
#todo

Comparison and Visualization

In [ ]:
#todo